In [6]:
import aiohttp
import asyncio
from bs4 import BeautifulSoup

/home/renangas/miniconda3/lib/python3.10/sre_parse.py:434: RuntimeWarning: coroutine 'main' was never awaited
  return list(dict.fromkeys(items))


In [18]:
result = ""

async def main():
    async with aiohttp.ClientSession() as session:
        async with session.get('https://emec.mec.gov.br/emec/consulta-curso/listar-historico-indicadores-curso/c1999930082674af6577f0c513f05a96/NDQ%3D/list/10') as resp:
            print(resp.status)
            result = await resp.text()
            soup = BeautifulSoup(result, 'html.parser')

            rows = soup.find('tbody').find_all('tr')

            # Find the row corresponding to the year 2014
            row_2014 = None
            for row in rows:
                if row.find('td', text='2014'):
                    row_2014 = row
                    break

            # Extract the ENADE measure for the year 2014
            if row_2014:
                enade_2014 = row_2014.find_all('td')[1].text
                print("ENADE measure for 2014:", enade_2014)
            else:
                print("Year 2014 data not found.")

In [19]:
await main()

200
ENADE measure for 2014: 5
